### Upsert in LangGraph

**Upsert** (a combination of "update" and "insert") is a database operation that allows for updating an existing record or inserting a new one if it does not already exist. In the context of **LangGraph**, an open-source stateful graph, upsert operations are crucial for efficiently managing and manipulating State within the graph. The upsert mechanism ensures that data consistency is maintained without the need for separate update and insert commands, thus simplifying the code and improving performance.

### Loading Stateful Graph in LangGraph

**Loading a stateful graph** in LangGraph involves initializing and managing the state of the graph, which includes the nodes, edges, and their properties. This process is essential for applications that require the persistence of graph state across different sessions and operations. Here’s a step-by-step outline of how this is typically done:

1. **Initialization**:
   - Start by initializing the LangGraph instance.
   - Load the graph's state if it exists or define a new State.
   - Start Mongo DB

2. **Data Import**:
   - Use the upsert operations to import data into the graph. This can be done by reading data from various sources (e.g., CSV files, databases) and inserting or updating the nodes and edges in the graph.
   - During this step, ensure that each node and edge is uniquely identified to prevent duplication and maintain data integrity.

3. **State Management**:
   - Implement mechanisms to track changes in the graph state. This can involve using timestamps, versioning, or change logs.
   - Store the state of the graph periodically or after significant updates to allow for recovery and rollback if necessary.

4. **Query and Analysis**:
   - Once the graph is loaded and stateful, use LangGraph's querying capabilities to extract insights, run algorithms, and perform analysis.
   - Ensure that the queries take into account the current state of the graph, reflecting the most recent updates.

5. **Persistence**:
   - Save the state of the graph to persistent storage. This can be achieved by serializing the graph state and storing it in a file or a database.
   - Ensure that the persistence mechanism supports efficient loading and saving to minimize downtime and performance impact.

6. **Recovery**:
   - Implement recovery procedures to restore the graph state from persistent storage in case of a failure or restart.
   - Ensure that the recovery process is robust and can handle partial or corrupted state files.

By following these steps, you can efficiently manage a stateful graph in LangGraph, leveraging its upsert capabilities to maintain a dynamic and consistent graph database. This approach is particularly useful for applications that require real-time updates and historical state tracking, such as social networks, recommendation systems,Human decision, and knowledge graphs.

In [ ]:
from typing import TypedDict, Optional
from bson import ObjectId
from langgraph.graph import StateGraph
from storage import MongoDBHandler

"""_id: use for mongo or any persistence key """


class GraphState(TypedDict):
    _id: Optional[str] = None
    question: Optional[str] = None
    strip: Optional[str] = None
    response: Optional[str] = None


workflow = StateGraph(GraphState)


In [ ]:
"""This is an entry point node that fetches data if there is an _id in the state."""


def decision_route_node(state):
    if _id := state.get("_id"):
        query = {"_id": ObjectId(_id)}
        db_handler = MongoDBHandler()
        loaded_dict = db_handler.load_dictionary('my_database', 'my_collection', query)
        return loaded_dict

In [ ]:
"""This is a finish point that stores the state before termination."""


def end_route_node(state):
    if not state.get('_id'):
        del state['_id']
    db_handler = MongoDBHandler()
    db_handler.insert_upsert_dictionary('my_database', 'my_collection', state)

In [ ]:

def strip_input_node(state):
    print("Skip this step when the state is populated in two ways: either through the strip in the input or when fetched from MongoDB.")
    question = state.get('question', '').strip()
    return {"strip": question}


def handle_greeting_node(state):
    return {"response": "Hello! How can I help you today?"}


def handle_search_node(state):
    question = state.get('question', '').strip()
    search_result = f"Search result for '{question}'"
    return {"response": search_result}


workflow.add_node("decision_route", decision_route_node)
workflow.add_node("strip_input", strip_input_node)
workflow.add_node("handle_greeting", handle_greeting_node)
workflow.add_node("handle_search", handle_search_node)
workflow.add_node("end_route", end_route_node)


def decide_next_node(state):
    return "handle_greeting" if state.get('strip') == "greeting" else "handle_search"


workflow.add_conditional_edges(
    "strip_input",
    decide_next_node,
    {
        "handle_greeting": "handle_greeting",
        "handle_search": "handle_search"
    }
)


def decide_route_node(state):
    """ You can manage your route when data loads based on the state in mongo."""
    if current_state := state.get('strip'):
        return decide_next_node(state)
    return "strip_input"


workflow.add_conditional_edges(
    "decision_route",
    decide_route_node,
    {
        "strip_input": "strip_input",
        "handle_search": "handle_search",
        "handle_greeting": "handle_greeting"
    }
)

workflow.set_entry_point("decision_route")
workflow.add_edge('handle_greeting', 'end_route')
workflow.add_edge('handle_search', 'end_route')
workflow.set_finish_point('end_route')
app = workflow.compile()

1. These examples illustrate a simple flow followed by storing the data in MongoDB.

In [ ]:
inputs = {"question": "       Hello, how are u?    "}
result = app.invoke(inputs)
inputs = {"question": "greeting"}
result = app.invoke(inputs)

2. These examples skip strip step: when loading data from the database or when manually provided in a dictionary.

In [ ]:
inputs = {"_id": "66534bc5ddaaf11b77c43295"}
result = app.invoke(inputs)

inputs = {"question": "       Hello, how are you?    ", "strip": "Hello, how are you?"}
result = app.invoke(inputs)